# Multi-Exchange Order Book Analysis

Analysis of order book data with individual publisher IDs preserved.

## Expected Data Sources

After running the multi-exchange pipeline, this notebook will show data from:
- **NASDAQ** (publisher_id 2) - XNAS.ITCH
- **NYSE** (publisher_id 9) - XNYS.PILLAR
- **IEX** (publisher_id 38) - IEXG.TOPS
- **NYSE Arca** (publisher_id 43) - ARCX.PILLAR
- **Cboe BZX** (publisher_id 5) - BATS.PITCH
- **NASDAQ BX** (publisher_id 3) - XBOS.ITCH
- **NASDAQ PSX** (publisher_id 4) - XPSX.ITCH

## Important Note

If you only see **1 exchange (NASDAQ)**, the binary file contains single-exchange data from before the multi-exchange implementation was added. Run the pipeline to generate new multi-exchange data.

In [1]:
import gzip
import struct
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timezone
from pathlib import Path
from collections import defaultdict

## Configuration

In [2]:
# File to analyze (will be created after running pipeline with multi-exchange)
FILE_PATH = 'sessions/CYPH-20251114.bin.gz'

# Exchange mapping (publisher_id to exchange name)
# Source: Databento API - client.metadata.list_publishers()
EXCHANGE_MAP = {
    1: 'CME Globex',
    2: 'NASDAQ',           # XNAS.ITCH
    3: 'NASDAQ BX',        # XBOS.ITCH
    4: 'NASDAQ PSX',       # XPSX.ITCH
    5: 'Cboe BZX',         # BATS.PITCH
    6: 'Cboe BYX',         # BATY.PITCH
    7: 'Cboe EDGA',        # EDGA.PITCH
    8: 'Cboe EDGX',        # EDGX.PITCH
    9: 'NYSE',             # XNYS.PILLAR
    10: 'NYSE National',   # XCIS.PILLAR
    11: 'NYSE American',   # XASE.PILLAR
    12: 'NYSE Chicago',    # XCHI.PILLAR
    15: 'MEMX',            # MEMX.MEMOIR
    16: 'MIAX Pearl',      # EPRL.DOM
    38: 'IEX',             # IEXG.TOPS
    43: 'NYSE Arca',       # ARCX.PILLAR
    95: 'EQUS Mini',       # EQUS.MINI
}

# Scaling factors
PRICE_SCALE = 100_000
SIZE_SCALE = 100
TIME_UNIT = 1_000_000

## Load Data

In [3]:
def read_binary_file(file_path):
    """Read and decode binary session file."""
    with gzip.open(file_path, 'rb') as f:
        data = f.read()
    
    # Read header
    magic, version, num_rows, initial_timestamp_us = struct.unpack('<4sHIQ', data[:18])
    
    if magic != b'TICK':
        raise ValueError(f"Invalid magic: {magic}")
    
    print(f"File: {Path(file_path).name}")
    print(f"Version: {version}")
    print(f"Rows: {num_rows:,}")
    
    # Read data rows
    rows = []
    offset = 18
    row_size = 64
    
    for i in range(num_rows):
        row_bytes = data[offset:offset + row_size]
        row = struct.unpack('<qBHIBBBqiHiIiiiiII', row_bytes)
        
        (delta_t, rtype, publisher_id, instrument_id, action, side, depth,
         price, size, flags, ts_in_delta, sequence,
         bid_px, ask_px, bid_sz, ask_sz, bid_ct, ask_ct) = row
        
        timestamp_us = initial_timestamp_us + delta_t
        time = datetime.fromtimestamp(timestamp_us / TIME_UNIT, tz=timezone.utc)
        
        rows.append({
            'timestamp': time,
            'timestamp_us': timestamp_us,
            'rtype': rtype,
            'publisher_id': publisher_id,
            'exchange': EXCHANGE_MAP.get(publisher_id, f'Unknown({publisher_id})'),
            'instrument_id': instrument_id,
            'action': chr(action) if action > 0 else ' ',
            'side': chr(side) if side > 0 else ' ',
            'depth': depth,
            'price': price / PRICE_SCALE,
            'size': size / SIZE_SCALE,
            'flags': flags,
            'ts_in_delta': ts_in_delta,
            'sequence': sequence,
            'bid_px': bid_px / PRICE_SCALE,
            'ask_px': ask_px / PRICE_SCALE,
            'bid_sz': bid_sz / SIZE_SCALE,
            'ask_sz': ask_sz / SIZE_SCALE,
            'bid_ct': bid_ct,
            'ask_ct': ask_ct,
        })
        
        offset += row_size
    
    return pd.DataFrame(rows)

# Load data
df = read_binary_file(FILE_PATH)
print(f"\nLoaded {len(df):,} rows")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Duration: {(df['timestamp'].max() - df['timestamp'].min()).total_seconds():.2f} seconds")

df.head()

File: CYPH-20251114.bin.gz
Version: 2
Rows: 1,722,425

Loaded 1,722,425 rows
Time range: 2025-11-14 05:28:14.714974+00:00 to 2025-11-14 23:59:37.115729+00:00
Duration: 66682.40 seconds


,timestamp,timestamp_us,rtype,publisher_id,exchange,instrument_id,action,side,depth,price,size,flags,ts_in_delta,sequence,bid_px,ask_px,bid_sz,ask_sz,bid_ct,ask_ct
0,2025-11-14 05:28:14.714974+00:00,1763098094714974,1,9,NYSE,21981,R,N,0,0.00,0.0,128,202576,2509,1.3,3.0,0.0,0.0,0,0
1,2025-11-14 05:36:56.315228+00:00,1763098616315228,1,43,NYSE Arca,21981,R,N,0,0.00,0.0,128,201349,971,1.3,3.0,0.0,0.0,0,0
2,2025-11-14 09:00:00.038099+00:00,1763110800038099,1,43,NYSE Arca,21981,T,N,0,1.78,151.0,0,200023,19662,1.3,3.0,0.0,0.0,0,0
3,2025-11-14 09:00:00.038099+00:00,1763110800038099,1,43,NYSE Arca,21981,T,N,0,1.78,1.0,0,200023,19663,1.3,3.0,0.0,0.0,0,0
4,2025-11-14 09:00:00.038099+00:00,1763110800038099,1,43,NYSE Arca,21981,T,N,0,1.78,6.0,0,200023,19664,1.3,3.0,0.0,0.0,0,0


In [ ]:
# Check for multi-exchange data
num_exchanges = df['exchange'].nunique()
unique_exchanges = sorted(df['exchange'].unique())

print("\n" + "="*80)
print("DATA TYPE DETECTION")
print("="*80)

if num_exchanges == 1:
    print(f"\n⚠️  WARNING: Only {num_exchanges} exchange detected!")
    print(f"   Exchange: {unique_exchanges[0]}")
    print(f"\n   This appears to be SINGLE-EXCHANGE data.")
    print(f"   To analyze multi-exchange data:")
    print(f"   1. Run: python scripts/main.py --date 2025-11-14")
    print(f"   2. Wait for pipeline to fetch from all 7 exchanges")
    print(f"   3. Re-run this notebook on the new binary file")
    print(f"\n   Expected result: 3-7 exchanges with ~800K-1.5M rows total")
else:
    print(f"\n✓ Multi-exchange data detected: {num_exchanges} exchanges")
    print(f"  Exchanges: {', '.join(unique_exchanges)}")

print("="*80)

## Exchange Distribution

In [ ]:
print("\n" + "="*80)
print("EXCHANGE DISTRIBUTION")
print("="*80)

exchange_counts = df['exchange'].value_counts()
print("\nQuotes by Exchange:")
for exchange, count in exchange_counts.items():
    pct = (count / len(df)) * 100
    pub_id = df[df['exchange'] == exchange]['publisher_id'].iloc[0]
    print(f"  {exchange:20s} (pub {pub_id:2d}): {count:8,} ({pct:5.1f}%)")

print(f"\nTotal exchanges: {len(exchange_counts)}")
print(f"Total quotes: {len(df):,}")

## Data Quality Check

In [ ]:
# Filter valid quotes (non-zero bid/ask)
df['spread'] = df['ask_px'] - df['bid_px']
df['mid_price'] = (df['bid_px'] + df['ask_px']) / 2
df['spread_pct'] = (df['spread'] / df['mid_price']) * 100

df_valid = df[(df['bid_px'] > 0) & (df['ask_px'] > 0)].copy()

print("\n" + "="*80)
print("DATA QUALITY")
print("="*80)
print(f"\nTotal rows: {len(df):,}")
print(f"Valid quotes (bid & ask > 0): {len(df_valid):,} ({len(df_valid)/len(df)*100:.1f}%)")
print(f"Invalid quotes: {len(df) - len(df_valid):,} ({(len(df) - len(df_valid))/len(df)*100:.1f}%)")

print("\nValid quotes by exchange:")
for exchange in df['exchange'].unique():
    total = len(df[df['exchange'] == exchange])
    valid = len(df_valid[df_valid['exchange'] == exchange])
    pct = (valid / total) * 100 if total > 0 else 0
    print(f"  {exchange:20s}: {valid:7,} / {total:7,} ({pct:5.1f}%)")

## Statistics by Exchange

In [ ]:
# Statistics for each exchange
print("\n" + "="*80)
print("STATISTICS BY EXCHANGE")
print("="*80)

exchange_stats = df_valid.groupby('exchange').agg({
    'timestamp': 'count',
    'bid_px': ['min', 'max', 'mean'],
    'ask_px': ['min', 'max', 'mean'],
    'spread': ['mean', 'median', 'std'],
    'spread_pct': ['mean', 'median'],
    'bid_sz': 'mean',
    'ask_sz': 'mean',
}).round(5)

exchange_stats.columns = ['_'.join(col).strip() for col in exchange_stats.columns.values]
exchange_stats = exchange_stats.rename(columns={'timestamp_count': 'updates'})
exchange_stats['update_pct'] = (exchange_stats['updates'] / len(df_valid) * 100).round(2)
exchange_stats = exchange_stats.sort_values('updates', ascending=False)

print(exchange_stats)

## Visualization 1: Exchange Distribution

In [ ]:
# Bar chart of quote distribution
exchange_counts_df = df_valid['exchange'].value_counts().reset_index()
exchange_counts_df.columns = ['exchange', 'count']
exchange_counts_df['percentage'] = (exchange_counts_df['count'] / len(df_valid) * 100).round(2)

fig = px.bar(
    exchange_counts_df,
    x='exchange',
    y='count',
    title='Quote Updates by Exchange (Multi-Exchange Data)',
    labels={'count': 'Number of Updates', 'exchange': 'Exchange'},
    text='percentage',
    color='count',
    color_continuous_scale='Viridis'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=500, showlegend=False)
fig.show()

## Visualization 2: Price Comparison Across Exchanges

In [ ]:
# Compare mid prices across exchanges over time
sample_n = max(1, len(df_valid) // 5000)
df_sample = df_valid.iloc[::sample_n].copy()

fig = go.Figure()

for exchange in sorted(df_valid['exchange'].unique()):
    df_ex = df_sample[df_sample['exchange'] == exchange]
    fig.add_trace(go.Scatter(
        x=df_ex['timestamp'],
        y=df_ex['mid_price'],
        mode='lines',
        name=exchange,
        line=dict(width=1.5)
    ))

fig.update_layout(
    title='Mid Price Comparison Across Exchanges',
    xaxis_title='Time',
    yaxis_title='Mid Price ($)',
    height=600,
    hovermode='x unified'
)

fig.show()

## Visualization 3: Spread Comparison

In [ ]:
# Box plot of spreads by exchange
fig = px.box(
    df_valid,
    x='exchange',
    y='spread',
    title='Bid-Ask Spread Distribution by Exchange',
    labels={'spread': 'Spread ($)', 'exchange': 'Exchange'},
    color='exchange'
)

fig.update_layout(height=500, showlegend=False)
fig.show()

## Visualization 4: Quote Activity Timeline

In [ ]:
# Resample to 1-minute intervals and count updates per exchange
df_valid_copy = df_valid.copy()
df_valid_copy['minute'] = df_valid_copy['timestamp'].dt.floor('1min')

activity = df_valid_copy.groupby(['minute', 'exchange']).size().reset_index(name='updates')

fig = px.line(
    activity,
    x='minute',
    y='updates',
    color='exchange',
    title='Quote Update Activity by Exchange (per minute)',
    labels={'minute': 'Time', 'updates': 'Updates per Minute', 'exchange': 'Exchange'}
)

fig.update_layout(height=600, hovermode='x unified')
fig.show()

## Visualization 5: Price Deviation Analysis

In [ ]:
# Calculate NBBO (best across all exchanges)
nbbo = df_valid.groupby('timestamp').agg({
    'bid_px': 'max',  # Best bid
    'ask_px': 'min',  # Best ask
}).reset_index()
nbbo['nbbo_mid'] = (nbbo['bid_px'] + nbbo['ask_px']) / 2

# Merge with original data
df_with_nbbo = df_valid.merge(nbbo[['timestamp', 'nbbo_mid']], on='timestamp', how='left')

# Calculate deviation from NBBO
df_with_nbbo['deviation_from_nbbo'] = df_with_nbbo['mid_price'] - df_with_nbbo['nbbo_mid']

# Plot deviation
fig = px.box(
    df_with_nbbo,
    x='exchange',
    y='deviation_from_nbbo',
    title='Price Deviation from NBBO by Exchange',
    labels={'deviation_from_nbbo': 'Deviation from NBBO ($)', 'exchange': 'Exchange'},
    color='exchange'
)

fig.add_hline(y=0, line_dash="dash", line_color="red", annotation_text="NBBO")
fig.update_layout(height=500, showlegend=False)
fig.show()

## Visualization 6: Order Book Depth by Exchange

In [ ]:
# Compare order book depth across exchanges
depth_stats = df_valid.groupby('exchange').agg({
    'bid_sz': 'mean',
    'ask_sz': 'mean'
}).reset_index()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=depth_stats['exchange'],
    y=depth_stats['bid_sz'],
    name='Avg Bid Size',
    marker_color='green'
))

fig.add_trace(go.Bar(
    x=depth_stats['exchange'],
    y=depth_stats['ask_sz'],
    name='Avg Ask Size',
    marker_color='red'
))

fig.update_layout(
    title='Average Order Book Depth by Exchange',
    xaxis_title='Exchange',
    yaxis_title='Average Size (shares)',
    barmode='group',
    height=500
)

fig.show()

## Summary Report

In [ ]:
print("\n" + "="*80)
print("MULTI-EXCHANGE ANALYSIS SUMMARY")
print("="*80)
print(f"\nFile: {FILE_PATH}")
print(f"Total rows: {len(df):,}")
print(f"Valid quotes: {len(df_valid):,} ({len(df_valid)/len(df)*100:.1f}%)")
print(f"Exchanges: {len(df_valid['exchange'].unique())}")
print(f"Duration: {(df['timestamp'].max() - df['timestamp'].min()).total_seconds():.2f} seconds")

print("\nExchange Breakdown:")
for exchange, count in df_valid['exchange'].value_counts().items():
    pct = (count / len(df_valid)) * 100
    pub_id = df_valid[df_valid['exchange'] == exchange]['publisher_id'].iloc[0]
    avg_spread = df_valid[df_valid['exchange'] == exchange]['spread'].mean()
    print(f"  {exchange:20s} (pub {pub_id:2d}): {count:7,} quotes ({pct:5.1f}%), avg spread: ${avg_spread:.5f}")

print("\nPrice Statistics:")
print(f"  NBBO Mid Price: ${nbbo['nbbo_mid'].mean():.5f} (avg)")
print(f"  Price Range: ${nbbo['nbbo_mid'].min():.5f} - ${nbbo['nbbo_mid'].max():.5f}")
print(f"  Best Spread: ${df_valid['spread'].min():.5f}")
print(f"  Worst Spread: ${df_valid['spread'].max():.5f}")
print(f"  Average Spread: ${df_valid['spread'].mean():.5f}")